In [1]:
import pandas as pd
import numpy as np

In [28]:
rets = pd.read_csv('Portfolios_Formed_on_ME_monthly_EW.csv', index_col = 0)
rets.index = pd.to_datetime(rets.index, format="%Y%m").to_period('M')
rets = rets[['Lo 20', 'Hi 20']]
rets.columns = ['SmallCap', 'LargeCap']
rets = rets.dropna()
rets = rets/100



In [29]:
rets.head(5)

,SmallCap,LargeCap
1926-07,-0.0057,0.0333
1926-08,0.0384,0.0233
1926-09,-0.0048,-0.0009
1926-10,-0.0329,-0.0295
1926-11,-0.0055,0.0316


In [30]:
#Annualized Return
number_of_observations = rets.shape[0]
annualized_returns = rets.add(1).prod() ** (12 / number_of_observations) - 1
annualized_returns

SmallCap    0.151977
LargeCap    0.098490
dtype: float64

In [32]:
#Annualized Volatility
annualized_volatility = rets.std()*np.sqrt(12)
annualized_volatility

SmallCap    0.336701
LargeCap    0.195116
dtype: float64

In [33]:
#1999 - 2015
limited_rets = rets["1999":"2015"]
number_of_obs = limited_rets.shape[0]
annualized_rets = (((1 + limited_rets).prod())**(12/number_of_obs)) - 1
annualized_rets


SmallCap    0.114426
LargeCap    0.062865
dtype: float64

In [34]:
annualized_volatility = limited_rets.std()*np.sqrt(12)
annualized_volatility

SmallCap    0.228894
LargeCap    0.172713
dtype: float64

In [45]:
wealth_index = 1000*(1+limited_rets["SmallCap"]).cumprod()
previous_peaks = wealth_index.cummax()
drawdown = (wealth_index - previous_peaks)/previous_peaks
drawdown.min()

-0.6248092431236988

In [46]:
drawdown.idxmin()

Period('2009-02', 'M')

In [47]:
wealth_index = 1000*(1+limited_rets["LargeCap"]).cumprod()
previous_peaks = wealth_index.cummax()
drawdown = (wealth_index - previous_peaks)/previous_peaks
drawdown.min()

-0.5527349978713653

In [48]:
drawdown.idxmin()

Period('2009-02', 'M')

In [55]:
hfi = pd.read_csv("edhec-hedgefundindices.csv", index_col=0, parse_dates=True)
hfi.index = hfi.index.to_period('M')
hfi = hfi/100


/var/folders/zc/mn8hy2xj3lv_nsv84bwc4gvm0000gn/T/ipykernel_55089/1433874712.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  hfi = pd.read_csv("edhec-hedgefundindices.csv", index_col=0, parse_dates=True)


In [56]:
hfi.head(5)

,Convertible Arbitrage,CTA Global,Distressed Securities,Emerging Markets,Equity Market Neutral,Event Driven,Fixed Income Arbitrage,Global Macro,Long/Short Equity,Merger Arbitrage,Relative Value,Short Selling,Funds Of Funds
date,,,,,,,,,,,,,
1997-01,0.0119,0.0393,0.0178,0.0791,0.0189,0.0213,0.0191,0.0573,0.0281,0.0150,0.0180,-0.0166,0.0317
1997-02,0.0123,0.0298,0.0122,0.0525,0.0101,0.0084,0.0122,0.0175,-0.0006,0.0034,0.0118,0.0426,0.0106
1997-03,0.0078,-0.0021,-0.0012,-0.0120,0.0016,-0.0023,0.0109,-0.0119,-0.0084,0.0060,0.0010,0.0778,-0.0077
1997-04,0.0086,-0.0170,0.0030,0.0119,0.0119,-0.0005,0.0130,0.0172,0.0084,-0.0001,0.0122,-0.0129,0.0009
1997-05,0.0156,-0.0015,0.0233,0.0315,0.0189,0.0346,0.0118,0.0108,0.0394,0.0197,0.0173,-0.0737,0.0275


In [57]:
hfi = hfi['2009':'2018']

In [58]:
hfi[hfi<0].std(ddof=0).sort_values()


Fixed Income Arbitrage    0.003321
Merger Arbitrage          0.003611
Global Macro              0.005075
Equity Market Neutral     0.005186
Relative Value            0.005711
Convertible Arbitrage     0.006681
Funds Of Funds            0.007948
Distressed Securities     0.009498
Event Driven              0.010135
CTA Global                0.010996
Long/Short Equity         0.012139
Emerging Markets          0.015276
Short Selling             0.021240
dtype: float64

In [59]:
def skewness(r):
    """
    Alternative to scipy.stats.skew()
    Computes the skewness of the supplied Series or DataFrame
    Returns a float or a Series
    """
    demeaned_r = r - r.mean()
    # use the population standard deviation, so set dof=0
    sigma_r = r.std(ddof=0)
    exp = (demeaned_r**3).mean()
    return exp/sigma_r**3
skewness(hfi).sort_values()

Equity Market Neutral    -0.896327
Funds Of Funds           -0.646908
Merger Arbitrage         -0.551065
Event Driven             -0.488821
Long/Short Equity        -0.463703
Distressed Securities    -0.254944
Emerging Markets          0.033123
CTA Global                0.052062
Relative Value            0.159953
Global Macro              0.348184
Short Selling             0.456518
Fixed Income Arbitrage    1.121453
Convertible Arbitrage     1.305911
dtype: float64

In [60]:
hfi = pd.read_csv("edhec-hedgefundindices.csv", index_col=0, parse_dates=True)
hfi.index = hfi.index.to_period('M')
hfi = hfi/100
hfi = hfi['2000':]


/var/folders/zc/mn8hy2xj3lv_nsv84bwc4gvm0000gn/T/ipykernel_55089/231561462.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  hfi = pd.read_csv("edhec-hedgefundindices.csv", index_col=0, parse_dates=True)


In [61]:
hfi.head(5)

,Convertible Arbitrage,CTA Global,Distressed Securities,Emerging Markets,Equity Market Neutral,Event Driven,Fixed Income Arbitrage,Global Macro,Long/Short Equity,Merger Arbitrage,Relative Value,Short Selling,Funds Of Funds
date,,,,,,,,,,,,,
2000-01,0.0227,0.0128,0.0088,0.0077,0.0075,0.0088,0.0041,0.0021,0.0075,0.0143,0.0173,0.0427,0.0169
2000-02,0.0267,-0.0022,0.0421,0.0528,0.0253,0.0346,0.0097,0.0408,0.0699,0.0239,0.0185,-0.1340,0.0666
2000-03,0.0243,-0.0138,0.0103,0.0318,0.0134,0.0069,-0.0061,-0.0104,0.0006,0.0131,0.0163,-0.0230,0.0039
2000-04,0.0223,-0.0241,-0.0101,-0.0541,0.0168,-0.0059,-0.0006,-0.0304,-0.0201,0.0188,0.0092,0.1028,-0.0269
2000-05,0.0149,0.0114,-0.0132,-0.0433,0.0062,-0.0034,0.0107,-0.0070,-0.0097,0.0146,0.0080,0.0704,-0.0122


In [63]:
def kurtosis(r):
    """
    Alternative to scipy.stats.kurtosis()
    Computes the kurtosis of the supplied Series or DataFrame
    Returns a float or a Series
    """
    demeaned_r = r - r.mean()
    # use the population standard deviation, so set dof=0
    sigma_r = r.std(ddof=0)
    exp = (demeaned_r**4).mean()
    return exp/sigma_r**4

In [64]:
kurtosis(hfi).sort_values()

CTA Global                 2.845022
Global Macro               3.653278
Short Selling              4.352758
Long/Short Equity          4.472589
Merger Arbitrage           4.520785
Event Driven               5.543444
Emerging Markets           5.941289
Distressed Securities      6.231720
Funds Of Funds             7.584725
Relative Value            12.731096
Equity Market Neutral     20.329621
Convertible Arbitrage     23.001449
Fixed Income Arbitrage    33.378729
dtype: float64